<a href="https://colab.research.google.com/github/Siddhant254/Deep-Learning-Concepts/blob/master/Hyperparameter_Tuning_using_KerasTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import *

In [3]:
import pandas as pd
df = pd.read_csv('/content/Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# Dependent variable
df['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [5]:
# Removing irrelevant columns
df = df.iloc[:,3:]
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
# Checking for null values
df.isnull().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [7]:
# Info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [8]:
# One hot Encoding
df = pd.get_dummies(df,columns=['Geography','Gender'],drop_first=True)
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


In [9]:
# Splitting data into Independent and Dependent Variable
x = df.drop(['Exited'],axis=1)
y = df[['Exited']]

In [10]:
# Balancing data
from imblearn.over_sampling import SMOTE
smote = SMOTE()
x_smote,y_smote = smote.fit_resample(x,y)
print(y.value_counts())
print(y_smote.value_counts())

Exited
0         7963
1         2037
dtype: int64
Exited
0         7963
1         7963
dtype: int64


In [11]:
# Split data into training and testing set
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_smote,y_smote,test_size=0.25,random_state=101)

In [12]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [13]:
x_train.shape

(11944, 11)

In [14]:
x_test.shape

(3982, 11)

In [15]:
# KERAS TUNER ---- Hyperparameter Tuning

In [16]:
!pip install keras-tuner --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 49.5 MB/s eta 0:00:00


In [18]:
# Importing keras tuner
from kerastuner.tuners import RandomSearch

In [19]:
def build_model(hp): # hp indicates hyperparameter
  model = Sequential()

  # Hyperparameter tuning
  counter = 0
  for i in range(hp.Int('no_hidden_layer',min_value=1,max_value=10)):
    if counter==0:
      model.add(Dense(hp.Int('no_neurons'+ str(i) , min_value=8 , max_value=256 , step=8),
                      activation = hp.Choice('activation_fun' + str(i) , values=['relu','leaky_relu','sigmoid'] ),input_dim=11))
      model.add(Dropout(hp.Choice('Dropout' + str(i) , values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7])))

    else:
      model.add(Dense(hp.Int('no_neurons'+ str(i) , min_value=8 , max_value=256 , step=8),
                      activation = hp.Choice('activation_fun' + str(i) , values=['relu','leaky_relu','sigmoid'] )))
      model.add(Dropout(hp.Choice('Dropout' + str(i) , values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7])))

    counter += 1
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('Optimizer',values=['adam','rmsprop','sgd']),
                loss='binary_crossentropy',metrics=['accuracy'])

  return model



In [20]:
# Initialize tuner
import kerastuner as kt

tuner = kt.RandomSearch(
    build_model,
    objective = 'val_accuracy',
    max_trials=5,
    directory = 'my_own_directory',project_name='Hyperparameter'
)

In [21]:
# Starting search
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 17s]
val_accuracy: 0.5007534027099609

Best val_accuracy So Far: 0.8151682615280151
Total elapsed time: 00h 01m 30s


In [22]:
 # best hyperparameter
 tuner.get_best_hyperparameters()[0].values

{'no_hidden_layer': 1,
 'no_neurons0': 176,
 'activation_fun0': 'relu',
 'Dropout0': 0.7,
 'Optimizer': 'rmsprop'}

In [23]:
model = tuner.get_best_models(num_models=1)[0]

In [24]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=5,validation_data=(x_test,y_test))

Epoch 6/100
374/374 [==============================] - 3s 6ms/step - loss: 0.4265 - accuracy: 0.8077 - val_loss: 0.4071 - val_accuracy: 0.8152
Epoch 7/100
374/374 [==============================] - 1s 4ms/step - loss: 0.4244 - accuracy: 0.8058 - val_loss: 0.4066 - val_accuracy: 0.8162
Epoch 8/100
374/374 [==============================] - 2s 4ms/step - loss: 0.4210 - accuracy: 0.8108 - val_loss: 0.4046 - val_accuracy: 0.8159
Epoch 9/100
374/374 [==============================] - 2s 5ms/step - loss: 0.4183 - accuracy: 0.8128 - val_loss: 0.4040 - val_accuracy: 0.8199
Epoch 10/100
374/374 [==============================] - 2s 7ms/step - loss: 0.4145 - accuracy: 0.8130 - val_loss: 0.4020 - val_accuracy: 0.8202
Epoch 11/100
374/374 [==============================] - 2s 4ms/step - loss: 0.4133 - accuracy: 0.8159 - val_loss: 0.3992 - val_accuracy: 0.8212
Epoch 12/100
374/374 [==============================] - 1s 4ms/step - loss: 0.4123 - accuracy: 0.8154 - val_loss: 0.3975 - val_accuracy: 0.8